In [ ]:
import numpy as np
import random
import imageio.v2 as io
import os
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter


def Saturation(img_rgb):
    saturation_range = (0.5, 1.5)  # Esempio: la saturazione sarà casuale tra 0.5 e 1.5
    saturation_factor = np.random.uniform(*saturation_range)
    saturated_image = img_rgb * saturation_factor
    saturated_image = np.clip(saturated_image, 0, 255).astype(np.uint8)
    return saturated_image

def Gaussian(img_rgb):
    numero_random = random.uniform(2, 6)
    img_rgb2 = gaussian_filter(img_rgb, sigma=(numero_random, numero_random, 0))
    return img_rgb2

def Channel(img_rgb):
    channel_order = np.random.permutation([0, 1, 2])
    image_swapped = img_rgb[:, :, channel_order]
    return image_swapped

def FlipDataset(img_rgb,img_rgbd,mask):
    img_rgb = np.flip(img_rgb, axis=1)
    img_rgbd = np.flip(img_rgbd, axis=1)
    mask = np.flip(mask, axis=1)
    return img_rgb,img_rgbd,mask

In [ ]:

path="../Datasets/DIODE_FULL - Copia (2)/"

percentuale=0.3
folder=os.listdir(path)
cnt=0

prova=[]
for scene in folder:
    scans=os.listdir(path+'/'+scene)
    for scan in scans:
        files=os.listdir(path+'/'+scene+'/'+scan)
        for file in files:
            if file.endswith(".png"):
                path_base=path+'/'+scene+'/'+scan+'/'+file[:-4]
                cnt+=1
                if(int(cnt%(1/percentuale))==0):
                    img_rgb = np.array(io.imread(path_base+'.png'))
                    img_rgbd = np.load(path_base+'_depth.npy')
                    mask = np.load(path_base+'_depth_mask.npy')
                    aug=''
                    var2 = random.random()
                    var3 = random.random()
                    var4 = random.random()
                    var5 = random.random()
                    if(var2>0.4):
                        img_rgb,img_rgbd,mask=FlipDataset(img_rgb,img_rgbd,mask)
                        aug=aug+'F'
                    if(var3>0.4):
                        img_rgb=Channel(img_rgb)
                        aug=aug+'C'
                    if(var4>0.4):
                        img_rgb=Gaussian(img_rgb)
                        aug=aug+'G'
                    if(var5>0.4):
                        img_rgb=Saturation(img_rgb)
                        aug=aug+'S'
                    print(aug)
                    # Salva l'immagine RGB
                    io.imwrite(path_base+'_'+aug+'.png', img_rgb)
                    # Salva l'immagine RGBD
                    np.save(path_base+'_depth_'+aug+'.npy', img_rgbd)
                    # Salva la maschera
                    np.save(path_base+'_depth_mask_'+aug+'.npy', mask)
